In [ ]:
import math
from openbabel import pybel

def squared_distance(coordsA, coordsB):
    """
    Find the squared distance between two 3-tuples
    """
    sqrdist = sum( (a-b)**2 for a, b in zip(coordsA, coordsB) )
    return sqrdist
    
def rmsd(allcoordsA, allcoordsB):
    """
    Finds the RMSD between two lists of 3-tuples.
    
    Parameters
    ----------
    
    allcoordsA : list
        Coordenates of the first structure. 
    allcoordsN : list
        Cornenates of the second structure. 
    
    Returns
    -------
    
    rmsd : float
        RMSD.
    """
    deviation = sum(squared_distance(atomA, atomB) for 
                    (atomA, atomB) in zip(allcoordsA, allcoordsB))
    return math.sqrt(deviation / float(len(allcoordsA)))

def compute_rmsd(mol1_path,mol2_path):
    """
    It calculates the symmetry-corrected RMSD between a ref an a que molecule. 
    Source: https://gist.github.com/baoilleach/974477
    
    Parameters
    ----------
    mol1_path : str
        Path to the PDB of the reference molecule. 
    mol2_path : str
        Path to the PDB of the query molecule. 
    
    Returns
    -------
    minrmsd : float
        Symmetry-corrected RMSD.
    """
    from openbabel import pybel
    # Loads the molecules from PDB files
    mol1 = next(pybel.readfile("pdb", mol1_path))
    mol2 = next(pybel.readfile("pdb", mol2_path))
    
    # Find automorphisms involving all atoms
    mappings = pybel.ob.vvpairUIntUInt()
    bitvec = pybel.ob.OBBitVec()
    lookup = []
    for i, atom in enumerate(mol1):
        bitvec.SetBitOn(i+1)
        lookup.append(i)
    success = pybel.ob.FindAutomorphisms(mol1.OBMol, mappings, bitvec)
    
    # Gets coordinates
    xtalcoords = [atom.coords for atom in mol1]
    posecoords = [atom.coords for atom in mol2]
    
    # Mapping and computes minimum RMSD  
    minrmsd = 999999999999
    for mapping in mappings:
        automorph_coords = [None] * len(xtalcoords)
        for x, y in mapping:
            automorph_coords[lookup.index(x)] = xtalcoords[lookup.index(y)]
        mapping_rmsd = rmsd(posecoords, automorph_coords)
        if mapping_rmsd < minrmsd:
            minrmsd = mapping_rmsd
    return minrmsd 

In [ ]:
def compute_tfd(QM_conf_id, FF_conf_id):
    """
    Computes the Torsion Fingerprint Deviation using RDKit for two structures. 
    
    Parameters
    ----------
    QM_conf_id : int
        Id of the reference structure. 
    FF_conf_id : int
        Id of the structure minimized with the Force Field. 
    
    Returns
    -------
    tfd : float
        Torsion Fingerprint Deviation
    """
    from rdkit.Chem import TorsionFingerprints, AllChem
    from rdkit import Chem

    QM_pdb_file = os.path.join(os.getcwd(), '{}/{}/ligand.pdb'
                                       .format(path, QM_conf_id))
    FF_pdb_file = os.path.join(os.getcwd(), '{}/{}/{}'
                                         .format(path, FF_conf_id, 'OpenMM_minimized.pdb'))

    QM_mol = Chem.rdmolfiles.MolFromPDBFile(QM_pdb_file)
    FF_mol = Chem.rdmolfiles.MolFromPDBFile(FF_pdb_file)
    newFF_mol = AllChem.AssignBondOrdersFromTemplate(QM_mol, FF_mol)
    tfd = TorsionFingerprints.GetTFDBetweenMolecules(QM_mol, newFF_mol)
    return tfd

In [ ]:
def parse_output_file(file):
    """
    Parsing the date from an output file after a PELEJob.
    """
    import re

    with open(file, 'r') as f:
        data = list()
        group = dict()
        for key, value in re.findall(r'(.*):\s*([\dE+-.]+)', f.read()):
            if key in group:
                data.append(group)
                group = dict()
            group[key] = value
        data.append(group)
    return data

In [ ]:
def _load_energies_dict(method, path):
    """
    It loads a python dictionary with the QM energies extracted from the SDF tag and 
    the energies after the minimization from the output file of a PELEJob o OpenMM minimization. 
    
    Parameters
    ----------
    
    mehtod : str
        Minimization method. 'PELE' or 'OPENMM'
    path : str
        Path to the running directory of the minimization output. 
    
    Returns
    -------
    energies_dict : dict
        Dictionary with the energies. 
    """
    energies_dict = {}
    mols_folders = os.listdir(path)
    with open('enes_QM.json') as f:
        ene_QM = json.load(f)
    for mol in mols_folders:
        try: 
            e_ini = ene_QM.get(mol)['Ene QM']
            if method == 'PELE':
                file_path = '{}/{}/PELE_output.txt'.format(path, mol)
                data = parse_output_file(file = file_path)
                e_fin = data[-2].get('ENERGY VACUUM + CONSTRAINTS')
            if method == 'OPENMM':
                file_path = '{}/{}/OpenMM_output.txt'.format(path, mol)
                data = open(file_path, 'r').read()
                e_fin = data.split(" ")[2]
            case = {'Ene QM': e_ini, 'Ene FF': e_fin}
            print(case)
            energies_dict[mol] = case
        except Exception as e: 
            print('Skipping: {}.format(e)')
            continue
    return energies_dict

## Compute TDF

In [ ]:
# Get all the different molecules in the set
path = 'openmm_openff-1.3_output'
molecules_ids = os.listdir(path)
molecules_ids = [mol_id for mol_id in molecules_ids if mol_id.isdigit()]

results = []
for mol_id in molecules_ids: 
    try:
        value = compute_tfd(mol_id, mol_id)
        results.append(value)
    except Exception as e:
        continue

In [ ]:
import numpy as np
np.savetxt("results/TFD_openmm13.csv", results, delimiter=",")

## Compute ddE and ddE vs. TDF

In [ ]:
import numpy as np
import os 
import json

# Get all the different molecules in the set
path = 'openmm_openff-1.3_output'
molecules_ids = os.listdir(path)
molecules_ids = [mol_id for mol_id in molecules_ids if mol_id.isdigit()]

# Load molecules and energies dictionaries
with open('data.json') as f:
    dict_mols = json.load(f)
energies_dict = _load_energies_dict('OPENMM', path)
molecules_names = []

for mol_id in molecules_ids:
    name = dict_mols.get(mol_id)['Name']
    molecules_names.append(name)
molecules_names = set(molecules_names)

In [ ]:
enes_method = []
tfds_method = []
for idx, name in enumerate(molecules_names):
    try:
        enes_QM, enes_FF, rmsds_mol, tfds_mol = ([] for i in range(4))

        QM_confs_ids = [key for key,values in dict_mols.items() \
                                            if values['Name'] == name and
                                            key in molecules_ids]
        FF_confs_ids = [key for key,values in dict_mols.items() \
                                            if values['Name'] == name and
                                            key in molecules_ids] 
        
        for QM_conf_id, FF_conf_id in zip(QM_confs_ids, FF_confs_ids):
            # Energies
            enes_QM.append(energies_dict.get(QM_conf_id)['Ene QM'])
            enes_FF.append(energies_dict.get(FF_conf_id)['Ene FF'])
            enes_QM = [float(ene) for ene in enes_QM]
            enes_FF = [float(ene) for ene in enes_FF]
            
            tfd = compute_tfd(QM_conf_id, FF_conf_id)
            tfds_mol.append(tfd)

        # compute relative energies against lowest E reference conformer
        lowest_ref_idx = enes_QM.index(min(enes_QM))
        rel_enes_QM = np.array(enes_QM) - enes_QM[lowest_ref_idx]
        rel_enes_FF = np.array(enes_FF) - enes_FF[lowest_ref_idx]

        # removes the 0th conformer
        rel_enes_QM = np.delete(rel_enes_QM, lowest_ref_idx)
        rel_enes_FF = np.delete(rel_enes_FF, lowest_ref_idx)
        tfds_mol = np.delete(tfds_mol, lowest_ref_idx)


        # Computes relative energy difference
        enes_mol = np.array(rel_enes_FF) - np.array(rel_enes_QM)

        # store results for this mol
        enes_method.append(enes_mol)
        tfds_method.append(tfds_mol)
        print('For molecule {}/{} ddE: {}, TFD: {}'.format(idx, len(molecules_names), enes_mol, tfds_mol))
    except Exception as e:
        print('Skipping: {}'.format(e))
        continue


In [ ]:
res1 = np.concatenate(enes_method).ravel()
res2 = np.concatenate(tfds_method).ravel()

In [ ]:
import numpy as np
np.savetxt("results/ddE_openmm13.csv", res1, delimiter=",")
np.savetxt("results/TFD1_openmm13.csv", res2, delimiter=",")

## Compute RMSD

In [ ]:
# Get all the different molecules in the set
import os 

path = 'openmm_openff-1.3_output'
molecules_ids = os.listdir(path)
molecules_ids = [mol_id for mol_id in molecules_ids if mol_id.isdigit()]

results = []
for idx, mol_id in enumerate(molecules_ids): 
    path1 = os.path.join(os.getcwd(), '{}/{}/ligand.pdb'
                                       .format(path, mol_id))
    path2 = os.path.join(os.getcwd(), '{}/{}/{}'
                                         .format(path, mol_id, 'OpenMM_minimized.pdb'))
    try:
        value = compute_rmsd(path1, path2)
        results.append(value)
        print('For molecule {}/{}, RMSD: {}'.format(idx, len(molecules_ids), value))
    except Exception as e:
        print('Skipping: {}'.format(e))
        continue

In [ ]:
import numpy as np
np.savetxt("results/RMSD_openmm13.csv", results, delimiter=",")